Starting from a 
* pre-trained model with a 
* pre-trained tokenizer

we perform finetuning on a negation task

In [108]:
%load_ext autoreload
%autoreload 2

import os 
import numpy as np
import random
from tqdm import tqdm

from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from torch import device, cuda, version

import apex

import dcc_splitter as splitter
import ner_training as trainer
import pandas as pd

import seaborn
import matplotlib.pyplot as plt

from collections import namedtuple

from transformers import AutoTokenizer, RobertaTokenizer, AutoModelForTokenClassification

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [110]:
device = device("cuda:0") if cuda.is_available() else device("cpu")

In [111]:
device

device(type='cpu')

In [112]:
dcc_dir = None
output_dir = None
skip_file = None
n_splits = 2
random_state = None
base_folder = "/media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels"
output_folder = "fine_tuned_token_classification"
mod_name = "robbert-v2-dutch-base" # "robbert-v2-dutch-base" # belabBERT_115k # bert-base-dutch
 

args = namedtuple
args.task = "negation" # experiencer, temporality
args.model_path = os.path.join(base_folder, mod_name)
args.model_type = "roberta" # bertje 
args.output_dir = os.path.join(base_folder, output_folder)
args.num_epochs = 1
args.eval_steps = 10 
args.lr = 5e-5
args.batch_size=16
args.gradient_accumulation_steps=1
args.block_size = 32 # block size determines inclusion 
args.save_model=False
args.bio=True
args.do_eval=False
args.do_write=False
args.bootstrap=False
args.do_print_class_report=False

random.seed(77)

In [ ]:
# args.block_size determines how many text snippets are  used for training, see ner_training.py lines 118--141
# obviously this is a code-design flaw that should be mended.
# the dataset loader should  include the id_begin_end in the output

In [113]:
# dcc-splitter for folds
dcc_splitter = splitter.DCCSplitter(dcc_dir, output_dir, skip_file, n_splits, random_state, write_to_file=False)
splits = dcc_splitter.split()

In [114]:
# load NER DCC set
dcc = pd.read_csv("../data/RobBERT/DCC_df.csv", 
                  sep="\t", 
                  skip_blank_lines=True, 
                  engine="python", 
                  encoding="latin-1",
                  on_bad_lines="warn", 
                  keep_default_na=False)

In [115]:
dcc.Negation.value_counts()

O             146395
NotNegated     15713
Negated         3017
Name: Negation, dtype: int64

In [116]:
Texts = dcc.groupby('Id').Word.apply(lambda x: " ".join(x))

In [117]:
dcc.loc[dcc.Id=='DL1795'][['Word', 'BIO', 'Negation', 'Experiencer', 'Temporality']].values

array([['Anamnese', 'O', 'O', 'O', 'O'],
       [':', 'O', 'O', 'O', 'O'],
       ['Op', 'O', 'O', 'O', 'O'],
       ['04', 'O', 'O', 'O', 'O'],
       ['.', 'O', 'O', 'O', 'O'],
       ['03', 'O', 'O', 'O', 'O'],
       ['.', 'O', 'O', 'O', 'O'],
       ['95', 'O', 'O', 'O', 'O'],
       ['liep', 'O', 'O', 'O', 'O'],
       ['patient', 'O', 'O', 'O', 'O'],
       ['een', 'O', 'O', 'O', 'O'],
       ['enkelletsel', 'B', 'NotNegated', 'Patient', 'Recent'],
       ['links', 'O', 'O', 'O', 'O'],
       ['op', 'O', 'O', 'O', 'O'],
       ['toen', 'O', 'O', 'O', 'O'],
       ['hij', 'O', 'O', 'O', 'O'],
       ['bij', 'O', 'O', 'O', 'O'],
       ['het', 'O', 'O', 'O', 'O'],
       ['korfballen', 'O', 'O', 'O', 'O'],
       ['na', 'O', 'O', 'O', 'O'],
       ['een', 'O', 'O', 'O', 'O'],
       ['sprong', 'O', 'O', 'O', 'O'],
       ['verkeerd', 'O', 'O', 'O', 'O'],
       ['neerkwam', 'O', 'O', 'O', 'O'],
       ['.', 'O', 'O', 'O', 'O'],
       ['Het', 'O', 'O', 'O', 'O'],
       ['letsel',

In [118]:
tag_ids = {'negation':{'B-Negated':0,'B-NotNegated':1,'I-Negated':2,'I-NotNegated':3},
          'temporality':{'B-Recent':0,'B-Historical':1,'B-Hypothetical':2,'I-Recent':3,'I-Historical':4,'I-Hypothetical':5},
          'experiencer':{'B-Patient':0,'B-Other':1,'I-Patient':2,'I-Other':3}}

tag2id = tag_ids[args.task]
tokenizer = AutoTokenizer.from_pretrained(args.model_path)

## Over all document sources

improvement: only output best model based on validation scores

In [119]:
# cycle through folds
scores = []
predlist = []
test_lists = []
loss_history = {}
for idx, fold in tqdm(enumerate(splits)):
    # re-init model for each fold, otherwise it keeps on training the same throughout all folds..
    token_model = AutoModelForTokenClassification.from_pretrained(args.model_path, num_labels = len(tag2id))
    
    train_list, test_list = fold['train'], fold['test']
    
    ## eval is optional (to gauge the best number of steps/epochs)
    eval_list = random.choices(train_list,k=int(len(train_list)/10)) if args.do_eval else []
    eval_dcc = dcc.loc[dcc.Id.isin(eval_list)]
    test_dcc = dcc.loc[dcc.Id.isin(test_list)]
    train_dcc = dcc.loc[(dcc.Id.isin(train_list)) & (~dcc.Id.isin(eval_list))]
    
    test_list = test_dcc.Id.tolist()
    eval_list = eval_dcc.Id.tolist()

    ###
    train_dataset = trainer.TextDatasetFromDataFrame(train_dcc, tokenizer, args) 
    test_dataset = trainer.TextDatasetFromDataFrame(test_dcc, tokenizer, args)
    eval_dataset = trainer.TextDatasetFromDataFrame(eval_dcc, tokenizer, args)
    
    args.do_print_class_report=False
    # Train on all document sources
    trained_model, eval_loss_history = trainer.train_model(model=token_model.to(device), 
                                                            tokenizer=tokenizer, 
                                                            train_dataset=train_dataset, 
                                                            eval_dataset=eval_dataset, 
                                                            tag2id=tag2id,
                                                            device=device, 
                                                            args=args,
                                                            max_grad_norm=1.0,
                                                            amp=False)
    args.do_print_class_report=True
    # Evaluate on all document sources
    f1, prec, rec, preds, truth = trainer.eval_model(model=trained_model, 
                                       tokenizer=tokenizer, 
                                       eval_dataset=test_dataset, 
                                       tag2id=tag2id, 
                                       device=device, 
                                       args=args, 
                                       return_pred=True)
    
    loss_history[idx]=eval_loss_history
    
    test_ids = ["_".join(t) for t in zip(test_dcc.Id, test_dcc.Begin, test_dcc.End)]
    scores.append({'fold': idx, 'f1': f1, 'precision': prec, 'recall': rec})
    predlist.append({'fold': idx, 'prediction': preds, 'truth': truth, 'ids': test_ids})
    test_lists.append(test_ids)
    

0it [00:00, ?it/s]Some weights of the model checkpoint at /media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels/robbert-v2-dutch-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint

RobertaConfig {
  "_name_or_path": "/media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels/robbert-v2-dutch-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [03:33<00:00,  1.27s/it]


Training finished, best model f = 0.000


1it [04:51, 291.22s/it]

...

RobertaConfig {
  "_name_or_path": "/media/koekiemonster/DATA-FAST/text_data/word_vectors_and_language_models/dutch/Medical/languagemodels/robbert-v2-dutch-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [03:33<00:00,  1.27s/it]


Training finished, best model f = 0.000


2it [09:42, 291.10s/it]

F1: 0.845 
              precision    recall  f1-score   support

     Negated       0.83      0.86      0.85       480
  NotNegated       0.81      0.89      0.85      2889

   micro avg       0.81      0.88      0.85      3369
   macro avg       0.82      0.87      0.85      3369
weighted avg       0.81      0.88      0.85      3369



In [120]:
predlist_prep = []
for foldnum, foldres in enumerate(predlist):
    cdx = 0
    ids = foldres['ids']
    for prs, trs in zip(foldres['prediction'], foldres['truth']):        
        for pr, tr in zip(prs, trs):
            tmp_dict={}
            if len(pr)==len(tr)==0:
                tmp_dict['fold'] = foldnum
                tmp_dict['entity_id'] = ids[cdx]
                tmp_dict['label'] = "n/a"
                tmp_dict['robbert'] = "n/a"
            elif len(pr)>0:
                tmp_dict['fold'] = foldnum
                tmp_dict['entity_id'] = ids[cdx]
                tmp_dict['label'] = tr
                tmp_dict['robbert'] = pr                
            else:
                raise ValueError("predictions are empty while truth is not")    
            cdx += 1
            predlist_prep.append(tmp_dict)
predlist_df = pd.DataFrame(predlist_prep)

In [124]:
predlist_df

,fold,entity_id,label,robbert
0,0,DL1667_O_O,B-NotNegated,B-NotNegated
1,0,DL1667_O_O,B-NotNegated,B-NotNegated
2,0,DL1667_O_O,B-NotNegated,B-NotNegated
3,0,DL1667_O_O,B-NotNegated,B-NotNegated
4,0,DL1667_O_O,B-Negated,B-Negated
...,...,...,...,...
10955,1,DL1448_O_O,B-NotNegated,B-NotNegated
10956,1,DL1448_O_O,B-Negated,B-Negated
10957,1,DL1448_O_O,I-Negated,I-Negated
10958,1,DL1448_O_O,B-NotNegated,B-NotNegated


In [121]:
test_scores = pd.DataFrame(scores)

In [70]:
if args.do_eval:
    dfl = []
    for i in range(2):
        df = pd.DataFrame(loss_history[i])
        df['fold']=i
        dfl.append(df)
    eval_history = pd.concat(dfl).reset_index()
    eval_history['step'] = eval_history['step'].astype(int)
    eval_history['fold'] = eval_history['fold'].astype(int)

    fig, ax = plt.subplots(ncols=3, figsize=(18,5))
    seaborn.lineplot(data=eval_history, x='step', y='f1', hue='epoch', ax=ax[0])
    seaborn.lineplot(data=eval_history, x='step', y='recall', hue='epoch', ax=ax[1])
    seaborn.lineplot(data=eval_history, x='step', y='precision', hue='epoch', ax=ax[2])

In [71]:
# micro-averaged scores
test_scores.round(2)

,fold,f1,precision,recall
0,0,0.78,0.73,0.82
1,1,0.77,0.73,0.81


In [72]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [73]:
# B-NotNegated, B-Negated
labmap = {'B-NotNegated': False, 'B-Negated': True, 'I-NotNegated': False, 'I-Negated': True}

manual_scores = []
confusion_matrix = []
for i in range(len(predlist)):
    # accuracy over all documents, flattened
    _predlist = [(labmap[t], 'B' if 'B-' in t else 'I') for l in predlist[i]['prediction'] for t in l if len(l)>0]
    _truthlist = [(labmap[t], 'B' if 'B-' in t else 'I') for l in predlist[i]['truth'] for t in l if len(l)>0]

    tr_c = []
    pr_c = []
    tr_r = []
    pr_r = []

    b_truth = []
    b_pred = []
    for _t,_p in zip(_truthlist, _predlist):
        if _t[1]==_p[1]=='B':
            tr_c.append(_t[0])
            pr_c.append(_p[0])
        tr_r.append(_t[0])
        pr_r.append(_p[0])

        b_truth.append(_t[1]=='B')
        b_pred.append(_p[1]=='B')

    tr_c, pr_c, tr_r, pr_r = np.array(tr_c), np.array(pr_c), np.array(tr_r), np.array(pr_r)
    b_truth, b_pred = np.array(b_truth), np.array(b_pred)

    TN_c = np.sum((pr_c==tr_c) & (pr_c==False))
    TP_c = np.sum((pr_c==tr_c) & (pr_c==True))
    FP_c = np.sum((pr_c!=tr_c) & (pr_c==True))
    FN_c = np.sum((pr_c!=tr_c) & (pr_c==False))

    TN_r = np.sum((pr_r==tr_r) & (pr_r==False))
    TP_r = np.sum((pr_r==tr_r) & (pr_r==True))
    FP_r = np.sum((pr_r!=tr_r) & (pr_r==True))
    FN_r = np.sum((pr_r!=tr_r) & (pr_r==False))

    TN_b = np.sum((b_pred==b_truth) & (b_pred==False))
    TP_b = np.sum((b_pred==b_truth) & (b_pred==True))
    FP_b = np.sum((b_pred!=b_truth) & (b_pred==True))
    FN_b = np.sum((b_pred!=b_truth) & (b_pred==False))


    # micro
    f1 = f1_score(tr_r, pr_r, average='micro')
    precision = precision_score(tr_r, pr_r, average='micro')
    recall = recall_score(tr_r, pr_r, average='micro')
    manual_scores.append({'list': 'raw', 
                          'fold': i, 
                          'focus': 'micro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # macro
    f1 = f1_score(tr_r, pr_r, average='macro')
    precision = precision_score(tr_r, pr_r, average='macro')
    recall = recall_score(tr_r, pr_r, average='macro')
    manual_scores.append({'list': 'raw', 
                          'fold': i, 
                          'focus': 'macro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # Negated
    f1 = f1_score(tr_r, pr_r)
    precision = precision_score(tr_r, pr_r)
    recall = recall_score(tr_r, pr_r)
    manual_scores.append({'list': 'raw', 
                          'fold': i, 
                          'focus': 'negated', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # NotNegated
    f1 = f1_score(~tr_r, ~pr_r)
    precision = precision_score(~tr_r, ~pr_r)
    recall = recall_score(~tr_r, ~pr_r)
    manual_scores.append({'list': 'raw',
                          'fold': i,
                          'focus': 'notnegated', 
                          'f1': f1,
                          'precision': precision,
                          'recall': recall})
    
    confusion_matrix.append({'list': 'raw', 'fold': i, 'TN': TN_r, 'TP': TP_r, 'FN': FN_r, 'FP': FP_r})    
    ######################################
    # micro
    f1 = f1_score(tr_c, pr_c, average='micro')
    precision = precision_score(tr_c, pr_c, average='micro')
    recall = recall_score(tr_c, pr_c, average='micro')
    manual_scores.append({'list': 'clean', 
                          'fold': i, 
                          'focus': 'micro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # macro
    f1 = f1_score(tr_c, pr_c, average='macro')
    precision = precision_score(tr_c, pr_c, average='macro')
    recall = recall_score(tr_c, pr_c, average='macro')
    manual_scores.append({'list': 'clean', 
                          'fold': i, 
                          'focus': 'macro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # Negated
    f1 = f1_score(tr_c, pr_c)
    precision = precision_score(tr_c, pr_c)
    recall = recall_score(tr_c, pr_c)
    manual_scores.append({'list': 'clean', 
                          'fold': i, 
                          'focus': 'negated', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # NotNegated
    f1 = f1_score(~tr_c, ~pr_c)
    precision = precision_score(~tr_c, ~pr_c)
    recall = recall_score(~tr_c, ~pr_c)
    manual_scores.append({'list': 'clean',
                          'fold': i,
                          'focus': 'notnegated', 
                          'f1': f1,
                          'precision': precision,
                          'recall': recall})
    
    confusion_matrix.append({'list': 'clean', 'fold': i, 'TN': TN_c, 'TP': TP_c, 'FN': FN_c, 'FP': FP_c})    
    ######################################
    # micro
    f1 = f1_score(b_truth, b_pred, average='micro')
    precision = precision_score(b_truth, b_pred, average='micro')
    recall = recall_score(b_truth, b_pred, average='micro')
    manual_scores.append({'list': 'B_I', 
                          'fold': i, 
                          'focus': 'micro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # macro
    f1 = f1_score(b_truth, b_pred, average='macro')
    precision = precision_score(b_truth, b_pred, average='macro')
    recall = recall_score(b_truth, b_pred, average='macro')
    manual_scores.append({'list': 'B_I', 
                          'fold': i, 
                          'focus': 'macro', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # Negated
    f1 = f1_score(b_truth, b_pred)
    precision = precision_score(b_truth, b_pred)
    recall = recall_score(b_truth, b_pred)
    manual_scores.append({'list': 'B_I', 
                          'fold': i, 
                          'focus': 'negated', 
                          'f1': f1, 
                          'precision': precision, 
                          'recall': recall})

    # NotNegated
    f1 = f1_score(~b_truth, ~b_pred)
    precision = precision_score(~b_truth, ~b_pred)
    recall = recall_score(~b_truth, ~b_pred)
    manual_scores.append({'list': 'B_I',
                          'fold': i,
                          'focus': 'notnegated', 
                          'f1': f1,
                          'precision': precision,
                          'recall': recall})
    
    confusion_matrix.append({'list': 'B_I', 'fold': i, 'TN': TN_b, 'TP': TP_b, 'FN': FN_b, 'FP': FP_b})    
    
manual_scores_df = pd.DataFrame(data=manual_scores)
confusion_matrix_df = pd.DataFrame(data=confusion_matrix)

In [74]:
manual_scores_df.groupby(['list', 'focus']).mean()

fold        f1  precision    recall
list  focus                                          
B_I   macro        0.5  0.877621   0.946072  0.842040
      micro        0.5  0.912538   0.912538  0.912538
      negated      0.5  0.942962   0.892145  1.000000
      notnegated   0.5  0.812281   1.000000  0.684080
clean macro        0.5  0.679240   0.943027  0.635483
      micro        0.5  0.890426   0.890426  0.890426
      negated      0.5  0.418976   1.000000  0.270966
      notnegated   0.5  0.939503   0.886054  1.000000
raw   macro        0.5  0.739898   0.934662  0.685291
      micro        0.5  0.897227   0.897227  0.897227
      negated      0.5  0.537619   0.977273  0.372170
      notnegated   0.5  0.942176   0.892052  0.998413

In [25]:
Texts['DL1932']

'Diagnose : status na glasverwonding in de linkerpols met uitval van de nervus ulnaris . Er bleek toen sprake van een doorsnijding van de arteria ulnaris , de nervus ulnaris , de flexor carpi ulnaris en van beide flexoren van de pink , allen aan de linkerzijde . Bij onderzoek is er sprake van een litteken aan de ulnovolaire zijde van de pols , verlengd naar de onderarm . Er is ook duidelijk sprake van een atrofie van de abductor pollicis met een krachtsvermindering tot 1 / 5 van de normale kracht . De abductor digiti quinti heeft ook een duidelijk krachtsverlies . Van de atrofie van de kleine handspieren is met name die van de adductor pollicis het meest opvallend .'

## Append to other results 

In [24]:
merged_result_file = '../results/merged_results.csv.gz'
results = pd.read_csv(merged_result_file)

In [25]:
entities = results.entity_id.str.replace(r"\_[0-9]+\_[0-9]+", "").unique()
len(entities)

/tmp/ipykernel_498924/2061256545.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  entities = results.entity_id.str.replace(r"\_[0-9]+\_[0-9]+", "").unique()


5365

In [32]:
results

,entity_id,category,label,bilstm,bilstm_cv,rule_based
0,DL1111_32_46,DL,not negated,not negated,not negated,not negated
1,DL1111_272_280,DL,not negated,not negated,not negated,not negated
2,DL1111_363_377,DL,not negated,not negated,not negated,not negated
3,DL1112_22_28,DL,negated,negated,negated,negated
4,DL1113_59_67,DL,not negated,not negated,not negated,not negated
...,...,...,...,...,...,...
12546,SP2118_862_876,SP,not negated,not negated,not negated,not negated
12547,SP2119_23_45,SP,negated,negated,negated,negated
12548,SP2120_3_23,SP,not negated,not negated,not negated,not negated
12549,SP2121_73_85,SP,not negated,not negated,not negated,not negated
